# Maths models for LLM Zoomcamp math competition

In [ ]:
# Installing necessary packages
! pip install -qU langchain-openai langchain

In [ ]:
# Necessary imports
import pandas as pd
import numpy as np
from langchain.prompts import PromptTemplate, OpenAI, ChatOpenAI
from dotenv import load_dotenv
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
import os
import re

## microsoft/rho-math-7b-interpreter-v0.1

## MathLLMs/MathCoder-CL-7B

## Qwen/Qwen2-Math-7B-Instruct

## deepseek-ai/deepseek-math-7b-rl